In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install wurlitzer

In [3]:
%load_ext wurlitzer

In [4]:
import logging
from pathlib import Path
from pathlib import Path
from ionbeam.core.config_parser import parse_config
from ionbeam.core.bases import *
from ionbeam.metadata.db import init_db
from ionbeam.sources.smart_citizen_kit.metadata import construct_sck_metadata
from datetime import datetime, timedelta, UTC
from ionbeam.core.time import TimeSpan

logging.basicConfig(level = logging.INFO)

config_file = Path("/Users/math/git/IonBeam_bundle/IonBeam/config").expanduser()
config, actions = parse_config(config_file,
                    environment  = "local",
                    sources = ["smart_citizen_kit"]
                    )

sck_source = actions[0]
devices = sck_source.get_ICHANGE_devices()

In [5]:
from IPython.display import display, JSON
JSON(devices[0])

<IPython.core.display.JSON object>

In [6]:
device = devices[0]
device_id = devices[0]["id"]
sensor = device['data']['sensors'][0]
sensor_id = sensor['id']
print(device_id, sensor_id)

time_span = TimeSpan.last(days = 10)

readings = sck_source.get_readings(device_id, sensor_id,  time_span)
sensor, readings

28 10


({'id': 10,
  'ancestry': None,
  'name': 'Battery SCK',
  'description': 'Custom Circuit',
  'unit': '%',
  'created_at': '2015-02-02T18:18:00Z',
  'updated_at': '2020-12-11T16:12:40Z',
  'uuid': 'c9ff2784-53a7-4a84-b0fc-90ecc7e313f9',
  'default_key': 'bat',
  'datasheet': None,
  'unit_definition': None,
  'measurement': {'id': 7,
   'name': 'battery',
   'description': 'The SCK remaining battery level in percentage.',
   'unit': None,
   'uuid': 'c5964926-c2d2-4714-98b5-18f84c6f95c1',
   'definition': None},
  'value': 71.7,
  'prev_value': 71.7,
  'last_reading_at': '2014-04-04T18:30:16Z',
  'tags': []},
 {'device_id': 28,
  'sensor_key': 'bat',
  'sensor_id': 10,
  'component_id': 9956,
  'rollup': '1s',
  'function': 'avg',
  'from': '2025-01-04T11:27:49Z',
  'to': '2025-01-14T11:27:49Z',
  'sample_size': 0,
  'readings': []})

In [43]:
device_id = 18146 
sensor_id = 113


# time_span = TimeSpan.last(hours = 1)
time_span = TimeSpan(start=datetime(2025, 1, 13, 11, 36, 55, 784173, tzinfo=UTC),
                     end=datetime(2025, 1, 14, 11, 26, 55, 784204, tzinfo=UTC))

device = sck_source.get_device(device_id)
print([s["last_reading_at"] for s in device['data']['sensors']])

sensor = next(s for s in device['data']['sensors'])

readings = sck_source.get_readings(device_id, sensor_id,  time_span)
sensor, readings

['2025-01-14T12:05:21Z', '2025-01-14T12:05:21Z', '2025-01-14T12:05:21Z', '2025-01-14T12:05:21Z', '2025-01-14T12:05:21Z', '2025-01-14T12:05:21Z', '2025-01-14T12:01:21Z', '2025-01-14T12:01:21Z', '2025-01-14T12:01:21Z', '2025-01-14T12:05:21Z', '2025-01-14T12:05:21Z', '2025-01-14T12:05:21Z', '2025-01-14T12:05:21Z', '2025-01-14T12:05:21Z']


({'id': 113,
  'ancestry': '111',
  'name': 'AMS CCS811 - TVOC',
  'description': 'Total Volatile Organic Compounds Digital Indoor Sensor',
  'unit': 'ppb',
  'created_at': '2019-03-21T16:43:37Z',
  'updated_at': '2019-03-21T16:43:37Z',
  'uuid': '0c2a1afc-dc08-4066-aacb-0bde6a3ae6f5',
  'default_key': 'tvoc',
  'datasheet': None,
  'unit_definition': None,
  'measurement': {'id': 47,
   'name': 'TVOC',
   'description': 'Total volatile organic compounds is a grouping of a wide range of organic chemical compounds to simplify reporting when these are present in ambient air or emissions. Many substances, such as natural gas, could be classified as volatile organic compounds (VOCs).',
   'unit': None,
   'uuid': 'c6f9a729-1782-4da1-adc9-e88b7143e45c',
   'definition': None},
  'value': 3604.0,
  'prev_value': 3862.0,
  'last_reading_at': '2025-01-14T12:05:21Z',
  'tags': []},
 {'device_id': 18146,
  'sensor_key': 'tvoc',
  'sensor_id': 113,
  'component_id': 87518207,
  'rollup': '1s',
  

In [8]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level = logging.INFO)

In [9]:
import pyfdb
import findlibs
import yaml
from pathlib import Path
import os
import pandas as pd
import pyodc
import shutil

fdb5_path = Path(findlibs.find("fdb5"))
metkit_path = Path(findlibs.find("metkit"))
print(f"Using fdb5 shared library from {fdb5_path}")
print(f"Using metkit shared library from {metkit_path}")

marsrequest = (metkit_path.parents[1] / "share/metkit/odb/marsrequest.yaml").resolve()
languages = (metkit_path.parents[1] / "share/metkit/language.yaml").resolve() 

print(f"\nContents of {marsrequest}:\n", marsrequest.open().read())

fields = " ".join(yaml.safe_load(languages.open().read())["_field"].keys())
print(f"Fields defined in {languages}:\n {fields}")

Using fdb5 shared library from /Users/math/micromamba/envs/ionbeam/lib/libfdb5.dylib
Using metkit shared library from /Users/math/micromamba/envs/ionbeam/lib/libmetkit.dylib

Contents of /Users/math/micromamba/envs/ionbeam/share/metkit/odb/marsrequest.yaml:
 ---
PLATFORM: platform
OBSERVATION_VARIABLE: observation_variable
EXTERNAL_ID: external_id
INTERNAL_ID: internal_id
AGGREGATION_TYPE: aggregation_type
VERSION: version
Fields defined in /Users/math/micromamba/envs/ionbeam/share/metkit/language.yaml:
 aggregation_type external_id internal_id observation_variable platform version class type stream expver dataset model repres obsgroup reportype levtype levelist param date hdate offsetdate fcmonth fcperiod time offsettime step anoffset reference number quantile domain frequency direction diagnostic iteration channel ident instrument method origin system


In [10]:
from pathlib import Path
from ionbeam.core.config_parser import parse_config
from ionbeam.core.bases import *
from IPython.display import display

config_file = Path("~/git/IonBeam-Deployment/config/ionbeam").expanduser()
config, actions = parse_config(config_file,
                    config_path = "./",
                    data_path = "../data/",
                    offline = True,
                    environment  = "local",
                    sources = ["smart_citizen_kit"]
                    )

ConfigError: /Users/math/git/IonBeam-Deployment/config/ionbeam does not exist!

In [ ]:
from uuid import UUID

id2action = {a.id : a for a in actions}
sources = [a for a in actions if isinstance(a, Source)]
processors = [a for a in actions if not isinstance(a, Source)]

chains = [[s,] for s in sources]
for a in processors:
    if not isinstance(a.match, UUID): 
        chains.append([str(a.match), a,])
        continue
    for c in chains:
        matches = a.match.int == c[-1].id.int
        if matches: c.append(a)
            

for c in chains:
    print(" --> ".join(str(a) for a in c))

In [ ]:
sources, stateless_actions, aggregators = [], [], []
for action in actions:
    if isinstance(action, Source):
        sources.append(action)
    elif isinstance(action, Aggregator):
        aggregators.append(action)
    else:
        stateless_actions.append(action)

print("Sources")
for i, a in enumerate(sources):
    display(a)

print("Aggregators")
for i, a in enumerate(aggregators):
    display(a)

print("Actions")
for i, a in enumerate(stateless_actions):
    display(a)

In [ ]:
sck_source = actions[0]
sck_source.value_columns

In [ ]:
import json
devices = sck_source.get_ICHANGE_devices()
print(json.dumps(devices[0], indent = 4))

In [ ]:
devices[0]["data"]["sensors"][0]

In [ ]:
from ionbeam.metadata.db import init_db

init_db(config.globals)

In [ ]:
from ionbeam.metadata import db
from shapely.geometry import Point

from ionbeam.sources.smart_citizen_kit.metadata import construct_sck_metadata

device = devices[0]
construct_sck_metadata(sck_source, device)

In [ ]:
from sqlalchemy.orm import Session
device = devices[0]
id = device["id"]


with Session(sck_source.globals.sql_engine) as session:
    stations = session.query(db.Station).all()
    station = stations[0]
    print("Retrieved station: ", station)
    j = station.as_json()
    print(station)
    print(station.sensors[1])
    print(station.sensors[1].properties)

print(json.dumps(j, indent = 4))

In [ ]:

for device in devices:
    construct_sck_metadata(sck_source, device)

In [ ]:
online = True
if online:
    source = sources[0]
    source.globals.offline = !online
    source.use_cache = False
    incoming_message_stream = source.generate()
    message = next(incoming_message_stream)
    message
else:
    p = Path('/Users/math/git/IonBeam-Deployment/data/inputs/meteotracker/MeteoTracker_62ae154f1d8e11061d4474b2.csv')
    message = FileMessage(
        metadata=MetaData(state='raw',
            source='meteotracker',
            filepath= p,
            variables = list(pd.read_csv(p).columns),             
            ))
message

In [ ]:
from ionbeam.core.bases import FinishMessage, FileMessage, MetaData
from ionbeam.aggregators import TimeAggregator

possible_actions = [a for a in actions if not isinstance(a, Source)]

message_history = []
while True:
    display(message)
    message_history.append(message)
    matching = [action for action in possible_actions if action.matches(message)]
    
    if not matching: 
        print("No more matches")
        break

    print("That matched with: ", [str(a) for a in matching])
    action = matching[0]

    # Special case for the TimeAggregator
    if action.__class__.__name__ == "TimeAggregator":
        list(action.process(message)) #need the list call here to pump the iterator to completion even if it doesn't return anything
        message = next(action.process(FinishMessage("We're done!")))
    else:
        message = next(action.process(message))
        

In [ ]:
m = message_history[-1]
m.data

In [ ]:

df = pd.DataFrame({"class" : ["rd", "rd"], "col1" : [1,2], "col2" : [1,2]})
pyodc.encode_odb(df, "test.odb")

with open("test.odb", "rb") as f:
    pyfdb.archive(f.read())

In [ ]:
import unicodedata as u

cs =  "¬!\"#£$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~°Ωαβγδθλμπωϕ£"

def unicode_info(c):
    d = dict(
        character = c,
        utf8 = c.encode("utf_8"),
        # utf16 = c.encode("utf_16"),
        name = u.name(c, "NO NAME"),
        category = u.category(c),
    )
    if u.decomposition(c): d["decomposition"] = u.decomposition(c)
    if not u.is_normalized('NFKD', c): d["normalized_name"] = u.name(u.normalize('NFKD', c))
    if not u.is_normalized('NFKD', c): print("\n".join(f"{k}: {v}" for k, v in d.items()) + "\n")

for c in cs: unicode_info(c)

In [ ]:
chr(0x1F600)

In [ ]:
s = "".join(chr(i) for i in range(100000))
with open("test.txt", "wb") as f:
    f.write(s.encode("utf-8", errors='ignore'))

In [ ]:
import unicodedata as u
u.name("·")